In [3]:
# import pandas as pd
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split

# df = pd.read_csv('data/speed_dating_raw.csv', encoding='latin-1')

# features = set(df.columns)
# features.remove('match')
# desc_features = list(features)

# x_train, x_test, y_train, y_test = train_test_split(df[desc_features], df['match'], test_size=.25, random_state=25)


# my_tree = DecisionTreeClassifier(max_depth=2)
# my_tree.fit(x_train, y_train)
# y_pred = my_tree.predict(x_test)
# acc = my_tree.score(x_test,y_test)
# print("Error: ", 1 - acc)





ValueError: could not convert string to float: 'Born in Iran'

In [103]:
import pandas as pd
# from geopy.geocoders import Nominatim
# import math
import numpy as np

cols = ['match', 'exphappy', 'samerace', 'hobby_diff_phys', 'hobby_diff_out', 'hobby_diff_in', 'same_goal', 'attr_diff', 'sinc_diff', 'intel_diff', 'fun_diff', 'amb_diff', 'income_diff', 'age_diff', 'same_career', 'confidence', 'imprace', 'date_freq', 'out_freq']
raw_df = pd.read_csv('data/speed_dating_raw.csv', encoding='latin-1')
df = pd.DataFrame(columns=cols)

hobbies = ['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga']

hobbies_phys = ['sports', 'exercise', 'hiking', 'yoga']
hobbies_out = ['dining', 'museums', 'concerts', 'clubbing', 'theater', 'movies', 'shopping']
hobbies_in = ['tvsports', 'art', 'gaming', 'reading', 'tv', 'music']

atts = ['attr', 'sinc', 'intel', 'fun', 'amb']
lookingfor = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1']
selfrate_a = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
selfrate_b = ['attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1']

def sum_diff(attrs,p_1,p_2):
    hobby_diff = 0
    for h in attrs:
        hobby_diff += abs(p_1[h] - p_2[h])
    return hobby_diff

def att_diff(p_1, p_2):
    all_diff = {}
    for i in range(5):
        diff = 0
        selfrate_a1 = p_1[selfrate_a[i]]
        selfrate_b1 = p_1[selfrate_b[i]]
        selfrate_a2 = p_2[selfrate_a[i]]
        selfrate_b2 = p_2[selfrate_b[i]]

        # if pd.isna(p_1[selfrate_a[i]]) and pd.isna(p_1[selfrate_b[i]]):
        #     selfrate_a1 = raw_df[selfrate_a[i]].mean()
        #     selfrate_b1 = raw_df[selfrate_b[i]].mean()
        if pd.isna(p_1[selfrate_a[i]]):
            selfrate_a1 = p_1[selfrate_b[i]]
        elif pd.isna(p_1[selfrate_b[i]]):
            selfrate_b1 = p_1[selfrate_a[i]]
        # if pd.isna(p_2[selfrate_a[i]]) and pd.isna(p_2[selfrate_b[i]]):
        #     selfrate_a2 = raw_df[selfrate_a[i]].mean()
        #     selfrate_b2 = raw_df[selfrate_b[i]].mean()
        if pd.isna(p_2[selfrate_a[i]]):
            selfrate_a2 = p_2[selfrate_b[i]]
        elif pd.isna(p_2[selfrate_b[i]]):
            selfrate_b2 = p_2[selfrate_a[i]]

        want_1 = p_1[lookingfor[i]]
        want_2 = p_2[lookingfor[i]]
        selfrate_1 = (selfrate_a1 + selfrate_b1) / 2
        selfrate_2 = (selfrate_a2 + selfrate_b2) / 2
        diff += abs(selfrate_1 - want_2)
        diff += abs(selfrate_2 - want_1)
        if pd.isna(p_1[lookingfor[i]]) and pd.isna(p_2[lookingfor[i]]): print('W')
        all_diff[atts[i]] = diff
    return all_diff

def plain_diff(key, p_1, p_2):
    item_1 = p_1[key]
    item_2 = p_2[key]
    if isinstance(item_1,str):
        item_1 = float(item_1.replace(',',''))
    if isinstance(item_2,str):
        item_2 = float(item_2.replace(',',''))
    if pd.isna(p_1[key]) or pd.isna(p_2[key]):
        return np.nan
    return abs(item_1 - item_2)

def dated(iid, pid):
    return len(raw_df.loc[(raw_df['iid'] == iid) & (raw_df['pid'] == pid)]) != 0

# def get_dist(p_1,p_2):
#     if isinstance(p_1['zipcode'], str):
#         zip_1 = int(p_1['zipcode'].replace(',',''))
#     else:
#         zip_1 = p_1['zipcode']
#     if isinstance(p_2['zipcode'], str):
#         zip_2 = int(p_2['zipcode'].replace(',',''))
#     else:
#         zip_2 = p_2['zipcode']
#     if pd.isna(zip_1) or pd.isna(zip_2) or (zip_1 == 0) or (zip_2 == 0):
#         return np.nan
#     geolocator = Nominatim(user_agent="geoapiExercises")
#     l_1 = geolocator.geocode(zip_1)
#     l_2 = geolocator.geocode(zip_2)
#     d = math.sqrt(((l_1.latitude - l_2.latitude) ** 2) + ((l_1.longitude - l_2.longitude) ** 2))
#     return d

def create_df():
    for iid in range(1,len(raw_df['iid'].unique())+1):
        for pid in raw_df.loc[raw_df['iid'] == iid]['pid'].unique():
            if dated(iid,pid):
                date = raw_df.loc[(raw_df['iid'] == iid) & (raw_df['pid'] == pid)].to_dict('records')[0]
                p_1 = raw_df.loc[raw_df['iid'] == iid].to_dict('records')[0]
                p_2 = raw_df.loc[raw_df['iid'] == pid].to_dict('records')[0]
                att_diffs = att_diff(p_1, p_2)
                # print(date['wave'],att_diffs)

                new_row = {
                    'match': date['match'],
                    'exphappy': date['exphappy'],
                    'samerace': date['samerace'],
                    'hobby_diff_phys': sum_diff(hobbies_phys,p_1,p_2),
                    'hobby_diff_out': sum_diff(hobbies_out, p_1, p_2),
                    'hobby_diff_in': sum_diff(hobbies_in, p_1, p_2),
                    'same_goal': int(p_1['goal'] == p_2['goal']),
                    'attr_diff': att_diffs['attr'],
                    'sinc_diff': att_diffs['sinc'],
                    'intel_diff': att_diffs['intel'],
                    'fun_diff': att_diffs['fun'],
                    'amb_diff': att_diffs['amb'],
                    'income_diff': plain_diff('income', p_1, p_2),
                    'age_diff': plain_diff('age', p_1, p_2),
                    'same_career': int(p_1['career_c'] == p_2['career_c']),
                    'confidence': ((((int(p_1['expnum']) / 20) + (int(p_2['expnum']) / 20)) / 2) if not (pd.isna(p_1['expnum']) or pd.isna(p_2['expnum'])) else p_1['expnum']),
                    'imprace': (((p_1['imprace'] + p_2['imprace']) / 2) if not (pd.isna(p_1['imprace']) or pd.isna(p_2['imprace'])) else p_1['imprace']),
                    'date_freq': p_1['date'],
                    'out_freq': p_1['go_out']
                }
                # print(new_row)
                df.loc[len(df)] = new_row
    return df

df = create_df()
df.to_csv('data/speed_dating.csv')


